In [2]:
import os
import glob
import pdal
import json
import rasterio
import numpy as np

In [7]:
INPUT_DIR = "lidar_data_raw"  # Folder containing multiple .laz files
OUTPUT_DIR = "lidar_data_processed"
RESOLUTION = 1.0  # meters

os.makedirs(OUTPUT_DIR, exist_ok=True)

def run_pdal_pipeline(pipeline_json):
    pipeline = pdal.Pipeline(json.dumps(pipeline_json))
    pipeline.execute()
    return pipeline

def create_dsm(input_laz, output_tif, resolution=1.0):
    pipeline = {
        "pipeline": [
            input_laz,
            {
                "type": "filters.smrf",  # Ground classification
                "ignore": "Classification[2:7]"
            },
            {
                "type": "writers.gdal",
                "filename": output_tif,
                "resolution": resolution,
                "output_type": "max",
                "gdaldriver": "GTiff"
            }
        ]
    }
    run_pdal_pipeline(pipeline)

def create_dtm(input_laz, output_tif, resolution=1.0):
    pipeline = {
        "pipeline": [
            input_laz,
            {
                "type": "filters.smrf"
            },
            {
                "type": "filters.range",
                "limits": "Classification[2:2]"
            },
            {
                "type": "writers.gdal",
                "filename": output_tif,
                "resolution": resolution,
                "output_type": "min",
                "gdaldriver": "GTiff"
            }
        ]
    }
    run_pdal_pipeline(pipeline)

def create_intensity_raster(input_laz, output_tif, resolution=1.0):
    pipeline = {
        "pipeline": [
            input_laz,
            {
                "type": "writers.gdal",
                "filename": output_tif,
                "resolution": resolution,
                "output_type": "mean",
                "dimension": "Intensity",
                "gdaldriver": "GTiff"
            }
        ]
    }
    run_pdal_pipeline(pipeline)

def create_ndsm(dsm_path, dtm_path, output_path):
    with rasterio.open(dsm_path) as dsm_src:
        dsm = dsm_src.read(1)
        meta = dsm_src.meta

    with rasterio.open(dtm_path) as dtm_src:
        dtm = dtm_src.read(1)

    ndsm = dsm - dtm
    ndsm[ndsm < 0] = 0

    meta.update(dtype=rasterio.float32)

    with rasterio.open(output_path, "w", **meta) as dst:
        dst.write(ndsm.astype(np.float32), 1)

def process_laz_file(input_laz_path):
    tile_name = os.path.splitext(os.path.basename(input_laz_path))[0]
    dsm_path = os.path.join(OUTPUT_DIR, f"{tile_name}_dsm.tif")
    dtm_path = os.path.join(OUTPUT_DIR, f"{tile_name}_dtm.tif")
    ndsm_path = os.path.join(OUTPUT_DIR, f"{tile_name}_ndsm.tif")
    intensity_path = os.path.join(OUTPUT_DIR, f"{tile_name}_intensity.tif")

    print(f"📍 Processing {tile_name}...")

    create_dsm(input_laz_path, dsm_path, RESOLUTION)
    create_dtm(input_laz_path, dtm_path, RESOLUTION)
    create_intensity_raster(input_laz_path, intensity_path, RESOLUTION)
    create_ndsm(dsm_path, dtm_path, ndsm_path)

    print(f"✅ Done with {tile_name}!\n")

def main():
    laz_files = glob.glob(os.path.join(INPUT_DIR, "*.laz"))
    if not laz_files:
        print("⚠️ No .laz files found in input directory.")
        return

    for laz_file in laz_files:
        process_laz_file(laz_file)

    print("🎉 All tiles processed!")

In [8]:
main()

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56922097...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56922097!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972110...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972110!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_57062124...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_57062124!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972106...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972106!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_57062119...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_57062119!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972128...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972128!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56922119...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56922119!

📍 Processing USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972115...
✅ Done with USGS_LPC_CA_AZ_FEMA_R9_Lidar_2017_D18_56972115!

📍 Proces